In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [15]:
standings_url = "https://fbref.com/en/comps/11/2022-2023/2022-2023-Serie-A-Stats"

In [16]:
data = requests.get(standings_url)

In [17]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [18]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [19]:
data = requests.get(team_urls[0])

In [20]:
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [21]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [22]:
data = requests.get(f"https://fbref.com{links[0]}")

In [23]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [24]:
shooting.head()

For Napoli                                                       \
         Date   Time     Comp        Round  Day Venue Result GF GA   
0  2022-08-15  18:30  Serie A  Matchweek 1  Mon  Away      W  5  2   
1  2022-08-21  18:30  Serie A  Matchweek 2  Sun  Home      W  4  0   
2  2022-08-28  20:45  Serie A  Matchweek 3  Sun  Away      D  0  0   
3  2022-08-31  20:45  Serie A  Matchweek 4  Wed  Home      D  1  1   
4  2022-09-03  20:45  Serie A  Matchweek 5  Sat  Away      W  2  1   

                  ... Standard               Expected                    \
        Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh G-xG   
0  Hellas Verona  ...     15.2  0.0  0     0      2.4  2.4    0.10  2.6   
1          Monza  ...     15.3  1.0  0     0      2.0  2.0    0.09  2.0   
2     Fiorentina  ...     14.7  1.0  0     0      1.7  1.7    0.13 -1.7   
3          Lecce  ...     17.7  0.0  0     0      1.7  1.7    0.09 -0.7   
4          Lazio  ...     16.1  0.0  0     0      2.1  2.1    0.11 -0.1   

          Unnamed: 25_level_0  
  np:G-xG        Match Report  
0     2.6        Match Report  
1     2.0        Match Report  
2    -1.7        Match Report  
3    -0.7        Match Report  
4    -0.1        Match Report  

[5 rows x 26 columns]

In [25]:
shooting.columns = shooting.columns.droplevel()

In [26]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [27]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-08-15,18:30,Serie A,Matchweek 1,Mon,Away,W,5,2,Hellas Verona,...,4-3-3,Michael Fabbri,Match Report,NaN,25,8,15.2,0.0,0,0
1,2022-08-21,18:30,Serie A,Matchweek 2,Sun,Home,W,4,0,Monza,...,4-3-3,Francesco Fourneau,Match Report,NaN,22,5,15.3,1.0,0,0
2,2022-08-28,20:45,Serie A,Matchweek 3,Sun,Away,D,0,0,Fiorentina,...,4-2-3-1,Livio Marinelli,Match Report,NaN,13,2,14.7,1.0,0,0
3,2022-08-31,20:45,Serie A,Matchweek 4,Wed,Home,D,1,1,Lecce,...,4-2-3-1,Matteo Marcenaro,Match Report,NaN,19,7,17.7,0.0,0,0
4,2022-09-03,20:45,Serie A,Matchweek 5,Sat,Away,W,2,1,Lazio,...,4-2-3-1,Simone Sozza,Match Report,NaN,19,7,16.1,0.0,0,0


In [95]:
years = list(range(2018, 2016, -1))
all_matches = []

In [96]:
standings_url = "https://fbref.com/en/comps/22/2018/2018-Major-League-Soccer-Stats"

In [97]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "MLS"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(7)

KeyError: "['FK'] not in index"

In [98]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, 'html.parser')

    # Paso 1: Obtener todos los enlaces de los equipos
    standings_tables = soup.select('table.stats_table')
    links = []
    for table in standings_tables:
        links.extend([l.get("href") for l in table.find_all('a')])

    # Paso 2: Filtrado de enlaces
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    # Obtener la URL de la temporada anterior
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    # Paso 3: Recorrer todos los URLs de los equipos
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        
        # Manejo de excepciones
        try:
            data = requests.get(team_url)
            matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
            soup = BeautifulSoup(data.text, 'html.parser')
            links = [l.get("href") for l in soup.find_all('a')]
            links = [l for l in links if l and 'all_comps/shooting/' in l]
            data = requests.get(f"https://fbref.com{links[0]}")
            shooting = pd.read_html(data.text, match="Shooting")[0]
            shooting.columns = shooting.columns.droplevel()
            
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt"]], on="Date")
            team_data = team_data[team_data["Comp"] == "MLS"]
            
            team_data["Season"] = year
            team_data["Team"] = team_name
            all_matches.append(team_data)
            
        except (ValueError, IndexError, AttributeError, requests.RequestException) as e:
            print(f"Error al procesar {team_url}. Error: {e}")
        
        # Paso 4: Optimización - espera para la siguiente solicitud
        time.sleep(7)

In [62]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Bundesliga"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(7)

In [99]:
len(all_matches)

491

In [100]:
match_df = pd.concat(all_matches)

In [101]:
match_df.columns = [c.lower() for c in match_df.columns]

In [102]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2018-03-10,19:00,MLS,Regular Season,Sat,Home,W,4,0,Portland Timbers,...,Match Report,NaN,20.0,9.0,18.4,1.0,0,0,2018,New York Red Bulls
1,2018-03-17,19:00,MLS,Regular Season,Sat,Away,L,0,1,Real Salt Lake,...,Match Report,NaN,9.0,4.0,15.2,0.0,0,0,2018,New York Red Bulls
2,2018-03-24,19:00,MLS,Regular Season,Sat,Home,W,3,0,Minnesota Utd,...,Match Report,NaN,21.0,9.0,17.8,0.0,0,0,2018,New York Red Bulls
3,2018-03-31,13:00,MLS,Regular Season,Sat,Away,L,3,4,Orlando City,...,Match Report,NaN,10.0,4.0,15.3,0.0,0,0,2018,New York Red Bulls
4,2018-04-14,13:00,MLS,Regular Season,Sat,Home,W,3,1,Montreal Impact,...,Match Report,NaN,18.0,10.0,16.5,1.0,0,0,2018,New York Red Bulls
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,2015-10-07,19:00,MLS,Regular Season,Wed,Home,D,0,0,FC Dallas,...,Match Report,NaN,8.0,2.0,NaN,NaN,0,0,2017,Vancouver Whitecaps FC
32,2015-10-14,20:00,MLS,Regular Season,Wed,Away,L,0,2,FC Dallas,...,Match Report,NaN,10.0,1.0,NaN,NaN,0,0,2017,Vancouver Whitecaps FC
33,2015-10-25,16:00,MLS,Regular Season,Sun,Home,W,3,0,Dynamo,...,Match Report,NaN,9.0,6.0,NaN,NaN,0,0,2017,Vancouver Whitecaps FC
34,2015-11-01,14:00,MLS,Conference Semifinals,Sun,Away,D,0,0,Portland Timbers,...,Match Report,Leg 1 of 2,NaN,NaN,NaN,NaN,0,0,2017,Vancouver Whitecaps FC


In [103]:
match_df.to_csv("matches-mls1718.csv")

In [25]:
match_df.to_csv("matches-pl-2122.csv")